In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the datasets
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
student_assessment = pd.read_csv('studentAssessment.csv')
student_info = pd.read_csv('studentInfo.csv')
student_registration = pd.read_csv('studentRegistration.csv')
student_vle = pd.read_csv('studentVle.csv')
vle = pd.read_csv('vle.csv')

# Step 2: Merge Relevant Data
# Merge studentAssessment with assessments to get module and presentation data
student_assessments = pd.merge(student_assessment, assessments, on='id_assessment')

# Merge with studentInfo to include demographic and educational background
student_data = pd.merge(student_assessments, student_info, on=['id_student', 'code_module', 'code_presentation'])

# Merge with studentVle to include sum of clicks data
student_data = pd.merge(student_data,
                        student_vle.groupby(['id_student', 'code_module', 'code_presentation'])['sum_click'].sum().reset_index(),
                        on=['id_student', 'code_module', 'code_presentation'],
                        how='left')

# Step 3: Select and Rename Columns
final_student_data = student_data[['id_student', 'highest_education', 'age_band', 'num_of_prev_attempts',
                                   'region', 'disability', 'code_module', 'score', 'sum_click', 'final_result']]

final_student_data.columns = ['Student_id', 'Highest_education', 'Age_band', 'Number_of_previous_attempts',
                              'Region', 'Disability', 'Code_module', 'Assessment_score', 'Student_sum_click',
                              'Student_final_result']

# Save to CSV
final_student_data.to_csv('Student_Assessment.csv', index=False)

# Step 4: Data Preprocessing on Student Assessment CSV File

# Reload the newly created CSV for preprocessing
student_assessment_df = pd.read_csv('Student_Assessment.csv')

# Check for Missing Values
missing_values = student_assessment_df.isnull().sum()
print(f"Missing values before handling:\n{missing_values}")

# Handling Missing Values
student_assessment_df.fillna(student_assessment_df.mean(numeric_only=True), inplace=True)  # For numeric columns
student_assessment_df.fillna('Unknown', inplace=True)  # For categorical columns

# Check again for missing values
missing_values_after = student_assessment_df.isnull().sum()
print(f"Missing values after handling:\n{missing_values_after}")

# Removing Duplicates
# Check for duplicates
duplicates = student_assessment_df.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")

# Remove duplicates
student_assessment_df.drop_duplicates(inplace=True)

# Check again for duplicates
duplicates_after = student_assessment_df.duplicated()
print(f"Number of duplicate rows after removal: {duplicates_after.sum()}")

# Handling Outliers
# Example: Removing rows where 'Assessment_score' is beyond 3 standard deviations
z_scores = np.abs((student_assessment_df['Assessment_score'] - student_assessment_df['Assessment_score'].mean()) / student_assessment_df['Assessment_score'].std())
student_assessment_df = student_assessment_df[z_scores < 3]

# Normalization/Standardization
# Standardizing numerical columns
scaler = StandardScaler()
numerical_cols = ['Assessment_score', 'Student_sum_click']
student_assessment_df[numerical_cols] = scaler.fit_transform(student_assessment_df[numerical_cols])

# Feature Selection
# Example: Dropping less relevant columns - can be customized based on analysis
features_to_drop = ['Region', 'Disability']  # Dropping for demonstration, adjust as necessary
student_assessment_df.drop(columns=features_to_drop, inplace=True)

# Save the final preprocessed data
student_assessment_df.to_csv('Student_Assessment_Preprocessed.csv', index=False)

print("Data preprocessing complete. Preprocessed data saved to 'Student_Assessment_Preprocessed.csv'.")


Missing values before handling:
Student_id                       0
Highest_education                0
Age_band                         0
Number_of_previous_attempts      0
Region                           0
Disability                       0
Code_module                      0
Assessment_score                 0
Student_sum_click              168
Student_final_result             0
dtype: int64
Missing values after handling:
Student_id                     0
Highest_education              0
Age_band                       0
Number_of_previous_attempts    0
Region                         0
Disability                     0
Code_module                    0
Assessment_score               0
Student_sum_click              0
Student_final_result           0
dtype: int64
Number of duplicate rows: 31752
Number of duplicate rows after removal: 0


TypeError: Could not convert ...?1869615785501890655157498063?...?84806861787383727060578630?785086?... to numeric